In [21]:
import os
import exifread
import numpy as np

def extract_gps_from_exif(file_path):
    """Extract GPS coordinates from an image file's EXIF data."""
    with open(file_path, 'rb') as f:
        tags = exifread.process_file(f)

    gps_latitude = tags.get('GPS GPSLatitude')
    gps_latitude_ref = tags.get('GPS GPSLatitudeRef')
    gps_longitude = tags.get('GPS GPSLongitude')
    gps_longitude_ref = tags.get('GPS GPSLongitudeRef')
    gps_altitude = tags.get('GPS GPSAltitude')
    
    if gps_latitude and gps_longitude:
        # Convert GPS coordinates to decimal
        lat = convert_to_decimal(gps_latitude, gps_latitude_ref)
        lon = convert_to_decimal(gps_longitude, gps_longitude_ref)
        alt = float(gps_altitude.values[0]) if gps_altitude else 0.0  # Use 0.0 if altitude is not available
        return lat, lon, alt
    else:
        return None, None, None

def convert_to_decimal(gps_coord, ref):
    """Convert GPS coordinates to decimal format."""
    degrees = gps_coord.values[0]
    minutes = gps_coord.values[1]
    seconds = gps_coord.values[2]
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if ref in ['S', 'W']:
        decimal = -decimal
    return decimal


def generate_geo_registration(folder_path: str, output_file: str, ext: str) -> None:
    """Iterate over image files in the folder, rename them sequentially, and write GPS data in ECEF format to a text file."""
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith((ext))])
    with open(output_file, 'w') as f:
        for i, file_name in enumerate(image_files):
            # Generate new file name
            new_file_name = f"frame_{i+1:05d}{ext}"
            old_file_path = os.path.join(folder_path, file_name)
            new_file_path = os.path.join(folder_path, new_file_name)
            
            # Extract GPS data
            lat, lon, alt = extract_gps_from_exif(old_file_path)
            
            if lat is not None and lon is not None:
                # Convert to ECEF (already adapted to model_aligner)                
                f.write(f"{new_file_name} {lat:.6f} {lon:.6f} {alt:.6f}\n")
            else:
                print(f"No GPS data found for {file_name}")

# Example usage
folder_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/raw_images"  # Replace with your folder path
output_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/ns_processed/colmap/sparse/geo-registration"        # Output file name
os.makedirs(output_dir, exist_ok = True)
ext = ".JPG"

output_file = os.path.join(output_dir, "geo-registration-list.txt")
generate_geo_registration(folder_path, output_file, ext)


In [2]:
from pyproj import Transformer
from colmap_parsing_utils import (
    qvec2rotmat,
    read_cameras_binary,
    read_images_binary,
    read_points3D_binary,
    write_images_binary,
    write_points3D_binary,
    write_cameras_binary,
)

import collections
import os
import struct
from typing import Dict, List
import numpy as np

CameraModel = collections.namedtuple("CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple("Camera", ["id", "model", "width", "height", "params"])
BaseImage = collections.namedtuple("Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])
Point3D = collections.namedtuple("Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])

def ecef_to_utm_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for ECEF to UTM conversion."""
    return Transformer.from_crs("EPSG:4978", f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84")

def ecef_to_utm_points(points_ecef: Dict[int, Point3D], transformer: Transformer) -> Dict[int, Point3D]:
    """Converts ECEF points to UTM coordinates."""

    points_utm = {}    
    for point_id, point in points_ecef.items():   
             
        x, y, z = point.xyz
        e, n, alt = transformer.transform(x, y, z)

        new_point = Point3D(
            id=point.id,
            xyz=np.array((e, n, alt)),
            rgb=point.rgb,
            error=point.error,
            image_ids=point.image_ids,
            point2D_idxs=point.point2D_idxs,
        )
        points_utm[point_id] = new_point
    return points_utm

# ????????????????????????????????? UTM는 camera pose 표현에는 적합하지 않은 것 같음. 따라서 ecef 사용하자.
def ecef_to_utm_images(images_ecef: Dict[int, BaseImage], transformer: Transformer) -> Dict[int, BaseImage]:
    """Converts image positions (camera centers) from ECEF to UTM coordinates."""
    images_utm = {}
    for image_id, image in images_ecef.items():
        # R = qvec2rotmat(image.qvec)
        # t = image.tvec  # Camera translation vector in ECEF
        x, y, z = image.tvec  # Camera translation vector in ECEF
        # camera_center = - R.T @ t
        e, n, alt = transformer.transform(x, y, z)
        new_image = BaseImage(
            id=image.id,
            qvec=image.qvec,
            tvec=np.array([e, n, alt]),  # Update translation vector
            camera_id=image.camera_id,
            name=image.name,
            xys=image.xys,
            point3D_ids=image.point3D_ids,
        )
        images_utm[image_id] = new_image
    return images_utm


def ecef_to_utm_cameras(cameras_ecef: Dict[int, Camera], transformer: Transformer) -> Dict[int, Camera]:
    """Transforms camera information (if required, e.g., extrinsics in params)."""
    # Most cameras have intrinsic data only, no transformation needed for params.
    # If extrinsic data is present in `params`, it can be transformed here.
    return cameras_ecef


# File paths
input_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/ns_processed/colmap/sparse/geo-registration/ecef"
output_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/ns_processed/colmap/sparse/geo-registration/utm"
os.makedirs(output_dir, exist_ok=True)

# Input files
input_points_path = os.path.join(input_dir, "points3D.bin")
input_images_path = os.path.join(input_dir, "images.bin")
input_cameras_path = os.path.join(input_dir, "cameras.bin")

# Output files
output_points_path = os.path.join(output_dir, "points3D.bin")
output_images_path = os.path.join(output_dir, "images.bin")
output_cameras_path = os.path.join(output_dir, "cameras.bin")

# Step 1: Read binary files
points3D_ecef = read_points3D_binary(input_points_path)
images_ecef = read_images_binary(input_images_path)
cameras_ecef = read_cameras_binary(input_cameras_path)

# Step 2: Create ECEF to UTM transformer
transformer = ecef_to_utm_transformer(zone_number=52, zone_letter="north")

# Step 3: Convert ECEF to UTM
points3D_utm = ecef_to_utm_points(points3D_ecef, transformer)
images_utm = ecef_to_utm_images(images_ecef, transformer)
cameras_utm = ecef_to_utm_cameras(cameras_ecef, transformer)

# Step 4: Write transformed data to binary files
write_points3D_binary(points3D_utm, output_points_path)
write_images_binary(images_utm, output_images_path)
write_cameras_binary(cameras_utm, output_cameras_path)

print(f"Converted {len(points3D_utm)} points from ECEF to UTM Zone 52N.")
print(f"Converted {len(images_utm)} images from ECEF to UTM Zone 52N.")
print(f"Converted {len(cameras_utm)} cameras.")


Converted 51423 points from ECEF to UTM Zone 52N.
Converted 85 images from ECEF to UTM Zone 52N.
Converted 1 cameras.


In [5]:
from osgeo import osr

# ECEF 좌표계 정의
source = osr.SpatialReference()
source.ImportFromProj4("+proj=geocent +datum=WGS84 +units=m +no_defs")

# UTM 좌표계 정의 (예: Zone 33N)
target = osr.SpatialReference()
target.ImportFromProj4("+proj=utm +zone=52 +datum=WGS84 +units=m +no_defs")

# 좌표 변환 객체 생성
transform = osr.CoordinateTransformation(source, target)

# ECEF 좌표 (X, Y, Z)
ecef_coords = (-3047511.45320001, 4051172.7114439, 3857827.98088647)

# 변환 수행
utm_coords = transform.TransformPoint(*ecef_coords)
print(f"UTM Easting: {utm_coords[0]}, Northing: {utm_coords[1]}, Altitude: {utm_coords[2]}")

# 318902.60516679112 4147421.4314777111 199.01728327572346

UTM Easting: 318903.473625324, Northing: 4147493.6933645126, Altitude: 143.62119787279516
